# Handcrafted Feature
In this section, we introduce point features and extraction. We can extract point features from raw data such as coordiantes, colors or viewpoint. of points are extracted from points and used to compare it with features of another point. Examples of feature usage are as follows.

- To perform REGISTRATION on two scene point clouds, feature-based point correspondence matching is used.
- One feature is calculated from the point cloud of an object and compared with the point cloud of another object.


When using a method that uses features, the method has the following flow. Feature extraction refers to the information that the point cloud has, such as the coordinate values and normals contained in the points of the point cloud, and calculates the information (features) that are useful for solving the task from this information. The features are then used in a postprocess that outputs the results of the desired task.

![processing](img/processing.png)

The background of this feature extraction and features is explained in the beginning of Section 4 of [Rusu, 2010]. The following is a quote from that text.

> In their native representation, points as defined in the concept of 3D mapping systems are simply represented using their Cartesian coordinates $x, y, z$, with respect to a given origin. Assuming that the origin of the coordinate system does not change over time, there could be two points $p_1$ and $p_2$ , acquired at $t_1$ and $t_2$ , having the same coordinates. Comparing these points however is an ill-posed problem, because even though they are equal with respect to some distance measure (e.g. Euclidean metric), they could be sampled on completely different surfaces, and thus represent totally different information when taken together with the other surrounding points in their vicinity. That is because there are no guarantees that the world has not changed between $t_1$ and $t_2$. Some acquisition devices might provide extra information for a sampled point, such as an intensity or surface remission value, or even a color, however that does not solve the problem completely and the comparison remains ambiguous.

> Applications which need to compare points for various reasons require better characteristics and metrics to be able to distinguish between geometric surfaces. The concept of a 3D point as a singular entity with Cartesian coordinates therefore disappears, and a new concept, that of local descriptor takes its place. The literature is abundant of different naming schemes describing the same conceptualization, such as shape descriptors or geometric features but for the remaining of this document they will be referred to as point feature representations.

In this section, we will explain each of the acquisition of point feature and its use in tasks for each point cloud processing method. Two types of point cloud processing methods will be explained in order from the top.

This section introduces point cloud handcrafted features below.
- PFH (Point Feature Histograms)
- FPFH (Fast Point Feature Histograms)
- PPF (Point Pair Feature)


In [1]:
%load_ext autoreload
%autoreload 2

## Detail about Handcrafted feature
<!-- Handcrafted featureはある点とその他いくつかの点を一組のグループとして扱い、グループ内の点の関係を特徴として使用します。例えば、Point Feature Histogram (PFH) [Rusu et al, 2008]は、任意の点pを中心としてその点と近傍点の関係性からヒストグラムを作成し、そのヒストグラムをpの特徴として提供しています。 -->


Handcrafted features treat a point and several other points as a group, and use the relationships between the points in the group as features. This relationship refers to the relative angle or distance between points. For example, Point Feature Histogram (PFH) [Rusu et al, 2008] creates a histogram from the relationships between a given point p and its neighbors, and provides the histogram as a feature of p.

![region](img/pfh.png)  
(Photo by [Rusu et al, 2009])

The calculated features are used for object detection and Registration. An example of Registration is shown below.


In [ ]:
# for registration
from tutlibs.sampling import voxel_grid_sampling
from tutlibs.registration import feature_ransac
from tutlibs.feature import PointFeatureHistograms as pfh
from tutlibs.normal_estimation import normal_estimation, normal_orientation

# for description
import numpy as np
from tutlibs.transformation import TransformationMatrix as tm
from tutlibs.transformation import Transformation as tr
from tutlibs.io import Points as io
from tutlibs.visualization import JupyterVisualizer as jv
from tutlibs.utils import single_color


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
# processing for target point cloud
target_xyz, _, _ = io.read("../data/redwood_indoor_1.ply")
target_ds_xyz = voxel_grid_sampling(target_xyz, 0.05)
target_ds_normal = normal_estimation(target_ds_xyz, k=15)
target_ds_normal = normal_orientation(target_ds_xyz, target_ds_normal)

# processing for source point cloud
source_xyz, _, _ = io.read("../data/redwood_indoor_2.ply")
source_xyz = tr.rotation(source_xyz, "x", np.pi / 5)
source_ds_xyz = voxel_grid_sampling(source_xyz, 0.05)
source_ds_normal = normal_estimation(source_ds_xyz, k=15)
source_ds_normal = normal_orientation(source_ds_xyz, source_ds_normal)

# extract Handcrafted feature
source_ds_pfh = pfh.compute(source_ds_xyz, source_ds_normal, 0.25)
target_ds_pfh = pfh.compute(target_ds_xyz, target_ds_normal, 0.25)

# feature matching
trans_mat = feature_ransac(
    source_ds_xyz, target_ds_xyz, source_ds_pfh, target_ds_pfh, 3, 150, 0.05
)
trans_source_xyz = tm.transformation(source_ds_xyz, trans_mat)

# visualize results
obj_source_points = jv.point(
    source_ds_xyz, single_color("#ff0000`", len(source_ds_xyz))
)
obj_trans_source_points = jv.point(
    trans_source_xyz, single_color("#00ff00", len(trans_source_xyz))
)
obj_target_points = jv.point(target_ds_xyz, single_color("#ffff00", len(target_ds_xyz)))
jv.display([obj_trans_source_points, obj_target_points, obj_source_points])


Output()

In the above implementation, we are performing PFH feature matching between point clouds, where the point cloud called source is predicted to partially overlap with the point cloud called target. Red indicates the initial location of source, yellow indicates the location of source after prediction, and green indicates the location of target.

Since PFH outputs histogram features based on the relative angle of the normal, it is more robust to differences in initial positions between point clouds than simpler methods that do not use features.

In the output result, you can see that the point cloud with the predicted transformation applied and the other point cloud are roughly overlapping. This result shows that PFH features are useful for matching in this example.

In the above implementation, we have dealt with a relatively simple example. There are several other methods proposed in Hand crafted features to deal with such problems as well.


## PFH (Point Feature Histograms)
PFH [Rusu et al, 2008]はクエリ点周りにある点を用いて、点間の法線相対角度と点間距離を使ったヒストグラムを表した特徴量である。

本subsectionで使用するコードは以下の通り。

In [2]:
from tutlibs.normal_estimation import normal_estimation
from tutlibs.feature import PointFeatureHistograms as pfh
from tutlibs.feature import pair_feature

from tutlibs.io import Points as io
import inspect

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
xyz, _, _ = io.read('../data/bunny_pc.ply')
normals = normal_estimation(xyz, 15)
pfhs = pfh.compute(xyz, normals)
print('PFH shape: {}'.format(pfhs.shape))

PFH shape: (3000, 125)


`pfh.compute`関数の出力が、点ごとの125次元PFHを表す。予め点群に法線情報が用意されている場合は法線を推定する必要がないため、`normal_estimation`関数は使わなくても良い。以下`pfh.compute`関数の中身を示す。

In [4]:
print(inspect.getsource(pfh.compute))

    @staticmethod
    def compute(xyz:np.ndarray, normals:np.ndarray, radius=0.03, div:int=5, normalization:bool=True) -> np.ndarray:
        """Compute Point Feature Histograms.
        Note: does not include the distance feature (reason -> https://pcl.readthedocs.io/projects/tutorials/en/latest/pfh_estimation.html#pfh-estimation)

        Args:
            xyz: xyz coords (N, 3)
            normals: normals (N, 3)
            radius: radius for nearest neighbor search
            div: number of subdivisions of value range for each feature.
            normalization: normalize each point histgram.
        Return:
            point feature histograms: (N, div**3)
        """
        knn_idxs, _, rnn_masks = radius_nearest_neighbors(xyz, xyz, r=radius, k=xyz.shape[0]) # shpae: (N, N), (N, N)

        # Get nearest neihobrs (get radius neihbors).
        num_pf = len(knn_idxs) # num_pf = N = xyz.shape[0]
        knn_xyzs = gather(xyz, knn_idxs) # shape: (N, N, 3)
        knn_normals = ga

PFHでは、点ごとに近傍点の座標値と法線を参照して特徴を計算し、その特徴に基づいてヒストグラムを作成する。上記実装では、

1. `radius_nearest_neighbors`関数によって、点群の各点と近傍点を取得する。下図の場合、点群の一点をクエリ点(赤点)として、クエリ点とクエリ点から一定距離内(点線の内)の近傍点(青点)を使用する。
2. for文で一点ごとにPFH算出を行う。一点のPFH算出には、1で得たクエリ点と近傍点を使用する。
   1. クエリ点と近傍点で二点の全組み合わせを作成する。下図では、辺の数だけ組み合わせが存在する。
   2. `pair_feature`関数を使用して、組み合わせごとの点間特徴を得る。
   3. 点間の特徴をヒストグラムで表現することで、PFHを取得する。ヒストグラムを作成する際は、前処理としてnormalization等を行う。

![nn](img/pfh_nn.png)  
[Rusu et al, 2009より]

次に、特徴を算出する`pair_feature`関数の中身を以下に示す。

In [5]:
print(inspect.getsource(pair_feature))

def pair_feature(xyz:np.ndarray, normals:np.ndarray, pair_idxs:np.ndarray):
    """Compute pair features.

    Args:
        xyz: xyz coords (M, 3)
        normals: normals (M, 3)
        pair_idxs: point pair indices (L, 2)
    Return:
        phi: cosine feature, value range is -1~1 (L)
        alpha: cosine feature, value range is -1~1 (L)
        theta: angle feature, value range is -pi/2~pi/2 (L)
        dists: distance feature (L)
    """
    # Get xyz and normal of points
    p1 = xyz[pair_idxs[:, 0]]
    n1 = normals[pair_idxs[:, 0]]
    p2 = xyz[pair_idxs[:, 1]]
    n2 = normals[pair_idxs[:, 1]]

    # Get vectors (pp) and distances (dists) between pt and ps
    pp = p2 - p1
    dists = np.linalg.norm(pp, ord=2, axis=1)

    # Get a mask to decide the source and target points.
    p1pp_angle = dot(n1, pp) / dists
    p2pp_angle = dot(n2, pp) / dists
    mask = np.arccos(np.fabs(p1pp_angle)) > np.arccos(np.fabs(p2pp_angle))

    # Decide source and target points.
    phi = p1pp

上記実装では、L個のペアとL個のペアで使用するクエリ点と近傍点のデータを入力する。出力としてPFHで使われるペアごとの特徴(ペア特徴)が出力される。ペア特徴は3つの法線相対角度phi($\phi$)、alpha($\alpha$)、theta($\theta$)と1つの相対距離dists($d$)の4種類である。注意点として、本実装ではPFHへ変換する際に$d$を使用しない。

2点をそれぞれ$p_s$、$p_t$として、点の法線と点を結ぶ線の角度が小さい方が$p_s$とする。この時、ペア特徴は以下の通り。

![pf](img/pfh_pf.png)  
[Rusu et al, 2010より修正]

$$
\phi=u \cdot \frac{\left(p_t-p_s\right)}{d} \\
\alpha = v \cdot n_t \\
\theta = \arctan(w \cdot n_t, u \cdot n_t) \\
d = \|p_t - p_s \|_2
$$



## Point Pair Feature (PPF)
PFH [Drost et al, 2010]は2点間の相対距離と点間の法線相対角度からなる特徴量である。具体的な処理については論文を参照。

本subsectionで使用するコードは以下の通り。

In [6]:
from tutlibs.normal_estimation import normal_estimation
from tutlibs.feature import PairPointFeature as ppf

from tutlibs.io import Points as io

In [ ]:
xyz, rgb, _ = io.read('../data/bunny_pc.ply')
normals = normal_estimation(xyz)
ppfs = ppf.compute(xyz, normals)
print('PPF shape: {}'.format(ppfs.shape))

## References
- [Rusu, R. B., N. Blodow, Z. C. Marton, and M. Beetz. 2008. “Aligning Point Cloud Views Using Persistent Feature Histograms.” In 2008 IEEE/RSJ International Conference on Intelligent Robots and Systems. IEEE. https://doi.org/10.1109/iros.2008.4650967.](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.391.5915&rep=rep1&type=pdf)
- [Rusu, Radu Bogdan, Nico Blodow, and Michael Beetz. 2009. “Fast Point Feature Histograms (FPFH) for 3D Registration.” In 2009 IEEE International Conference on Robotics and Automation, 3212–17.](https://www.cvl.iis.u-tokyo.ac.jp/class2016/2016w/papers/6.3DdataProcessing/Rusu_FPFH_ICRA2009.pdf)
- [Drost, Bertram, Markus Ulrich, Nassir Navab, and Slobodan Ilic. 2010. “Model Globally, Match Locally: Efficient and Robust 3D Object Recognition.” In 2010 IEEE Computer Society Conference on Computer Vision and Pattern Recognition. IEEE. https://doi.org/10.1109/cvpr.2010.5540108.](http://campar.in.tum.de/pub/drost2010CVPR/drost2010CVPR.pdf)
- [Rusu, R.B. Semantic 3D Object Maps for Everyday Manipulation in Human Living Environments. Künstl Intell 24, 345–348 (2010). https://doi.org/10.1007/s13218-010-0059-6](https://link.springer.com/article/10.1007/s13218-010-0059-6#citeas)
- [Rusu, Radu Bogdan, Zoltan Csaba Marton, Nico Blodow, and Michael Beetz. 2008. “Learning Informative Point Classes for the Acquisition of Object Model Maps.” In 2008 10th International Conference on Control, Automation, Robotics and Vision. IEEE. https://doi.org/10.1109/icarcv.2008.4795593.](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.584.2647&rep=rep1&type=pdf)